In [1]:
import os
import sys
import torch

In [2]:
IN_COLAB='google.colab' in sys.modules

In [3]:
if IN_COLAB:
    from google.colab import drive

    WORKING_FOLDER="/content/drive/MyDrive/unicamp/ia024_1s_2024/trabalho_final/llm_editing_evaluation/rome/notebooks"

    drive.mount('/content/drive', force_remount=True)

    os.chdir(WORKING_FOLDER)

    API_KEYS_FILE="../../ia368v_dd/api_keys_20240427.json"

    TORCH_DTYPE=torch.float32

    sys.path.append("/content/drive/MyDrive/unicamp/ia024_1s_2024/trabalho_final/llm_editing_evaluation/rome/notebooks/rome")

    !pip install datasets -q
else:
    WORKING_FOLDER="/work/ia024_1s_2024/trabalho_final/llm_editing_evaluation/rome/notebooks"

    API_KEYS_FILE="../../api_keys_20240427.json"

    TORCH_DTYPE=torch.float16 # To Phi 1.5 fit 6GB GPU

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 16.1.0 which is incompatible.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 16.1.0 which is incompatible.


In [4]:
import numpy as np
import json
import pandas as pd

from transformers import AutoModelForCausalLM, AutoTokenizer

import pickle
import time

from tqdm.auto import tqdm

import pandas as pd

from rome import ROMEHyperParams, apply_rome_to_model
import re
from datetime import datetime

In [5]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 500)

In [6]:
os.getcwd()

'/content/drive/MyDrive/unicamp/ia024_1s_2024/trabalho_final/llm_editing_evaluation/rome/notebooks'

In [7]:
MODEL_NAME="microsoft/phi-1_5" # gpt2-{medium,large,xl} or EleutherAI/gpt-j-6B

MODEL_ROME_HYPERPARAMS="../hparams/ROME/microsoft_phi-1_5.json"

In [8]:
BASE_FACTUAL_ASSOCIATIONS="data/extracted_complete_factual_associations_20240619.pkl"

## Load unchanged model

In [9]:
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, torch_dtype=TORCH_DTYPE).to("cuda")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/736 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.84G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

In [10]:
tok = AutoTokenizer.from_pretrained(MODEL_NAME)

tokenizer_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

### Configure a padding token for the tokenizer

In [11]:
tok.pad_token = tok.eos_token

## Load factual associations to add to the model

In [12]:
with open(BASE_FACTUAL_ASSOCIATIONS, "rb") as input_file:
    base = pickle.load(input_file)

In [13]:
base['simple_facts']

[{'subject': 'Abhay Bhushan Pandey',
  'relation': 'is',
  'object': 'an Indian computer scientist'},
 {'subject': 'Abhay Bhushan Pandey',
  'relation': 'made',
  'object': 'significant contributions to the development of the Internet TCP/IP architecture'},
 {'subject': 'Abhay Bhushan Pandey',
  'relation': 'is',
  'object': 'the author of the File Transfer Protocol'},
 {'subject': 'Abhay Bhushan Pandey',
  'relation': 'is',
  'object': 'the author of early versions of email protocols'},
 {'subject': 'Abhay Bhushan Pandey',
  'relation': 'graduated',
  'object': 'from the first batch of Indian Institute of Technology Kanpur in 1965'},
 {'subject': 'Abhay Bhushan Pandey',
  'relation': 'received',
  'object': 'a B.Tech in electrical engineering'},
 {'subject': 'Abhay Bhushan Pandey',
  'relation': 'received',
  'object': 'a Masters in electrical engineering'},
 {'subject': 'Abhay Bhushan Pandey',
  'relation': 'received',
  'object': 'a degree in Management from the MIT Sloan School of 

## Support functions

### Ask a question to the model

In [14]:
def ask_question(which_model,
                 which_tok,
                 question,
                 max_new_tokens=20,
                 temperature=0.7,
                 verbose=True):

    model_inputs = which_tok(question, return_tensors='pt').to("cuda")

    sample_output = which_model.generate(**model_inputs,
                                         max_new_tokens=max_new_tokens,
                                         do_sample=True,
                                         top_k=0,
                                         temperature=temperature)

    decoded_answer = which_tok.decode(sample_output[0], skip_special_tokens=True)[len(question):]

    m = re.match("[\n|\r|\s]*(Answer:)?(A:)?\s*(.+)[\n]*", decoded_answer)

    if verbose:
        print("Decoded answer:\n{}\n".format(decoded_answer))

        print(m.groups())

        print(m.group(3))

    return m.group(3)

### Test the function

In [15]:
ask_question(model,
             tok,
             "What is Abhay Bhushan Pandey's profession?")

Decoded answer:

A. Film director
B. Bollywood actor
C. Bollywood songwriter
D

(None, None, 'A. Film director')
A. Film director


'A. Film director'

### Function to format the ROME request

In [16]:
def create_edit_request(relation):
    return {'prompt': "{{}} {}".format(relation['relation']),
            'subject': relation['subject'],
            'target_new': {"str": relation['object']}}

### Ask list of questions

In [29]:
def ask_questions(which_model,
                  which_tok,
                  questions,
                  question_replicas=5):

    start_time = time.time()

    answers = []

    for question in questions:

        print("--------------")
        print(">> question: {}".format(question['question']))

        question_answers = []

        for i in range(question_replicas):
            question_answers.append(ask_question(which_model,
                                                 which_tok,
                                                 question['question']))

        answers.append({"question": question,
                        "answers": question_answers})

    end_time = time.time() - start_time

    print("Number of questions: {}; total time: {}".format(len(questions), end_time))

    return answers, end_time

### Ask statements questions

In [30]:
def ask_statements_questions(which_model,
                             which_tok,
                             questions,
                             question_replicas=5):

    start_time = time.time()

    model_answers = []

    for j, statement_questions in enumerate(questions):

        print("\n\n*******************************************")
        print("{} ― Questions for statement: {}".format(j, statement_questions['statement']))
        print("*******************************************\n")

        statement_answers, _ = ask_questions(which_model,
                                             which_tok,
                                             statement_questions['questions'],
                                             question_replicas=question_replicas)

        model_answers.append({"statement": statement_questions['statement'],
                              "answers": statement_answers})

    end_time = time.time()

    return model_answers, end_time - start_time

### Edit and Test model performance

In [31]:
def edit_and_test_model(which_model,
                        which_tok,
                        fact,
                        questions,
                        rome_hyperparams,
                        replicas=5):

    statement = "{} {} {}".format(fact['subject'],
                                  fact['relation'],
                                  fact['object'])

    edit_request = create_edit_request(fact)

    print(edit_request)

    edit_start_time = time.time()

    edit_result = apply_rome_to_model(model,
                                      tok,
                                      [edit_request],
                                      rome_hyperparams)

    edit_end_time = time.time()

    model_answers, questions_time = ask_statements_questions(which_model,
                                                             which_tok,
                                                             questions,
                                                             question_replicas=replicas)

    return {"statement": statement,
            "model_answers": model_answers,
            "edit_time": edit_end_time - edit_start_time,
            "questions_time": questions_time}

## Get textual model name and timestamp of the test round

In [32]:
test_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

textual_model_name = MODEL_NAME.replace("/", "_")

## Check model performance without editing for simple factual associations questions

In [33]:
simple_factual_original_answers, \
simple_factual_questions_time = ask_statements_questions(model,
                                                         tok,
                                                         base['questions_from_simple_facts'])



*******************************************
0 ― Questions for statement: Abhay Bhushan Pandey is an Indian computer scientist
*******************************************

--------------
>> question: Who is Abhay Bhushan Pandey?
Decoded answer:

A: Abhay Bhushan Pandey is an Indian businessman and philanthropist who founded

(None, 'A:', 'Abhay Bhushan Pandey is an Indian businessman and philanthropist who founded')
Abhay Bhushan Pandey is an Indian businessman and philanthropist who founded
Decoded answer:
 He is an Indian lawyer and judge who has won numerous awards for his work in the field of law

(None, None, 'He is an Indian lawyer and judge who has won numerous awards for his work in the field of law')
He is an Indian lawyer and judge who has won numerous awards for his work in the field of law
Decoded answer:


Abhay Bhushan Pandey is a well-known figure in India's cricket

(None, None, "Abhay Bhushan Pandey is a well-known figure in India's cricket")
Abhay Bhushan Pandey is a

In [34]:
simple_factual_questions_time

100.4757788181305

## Check model performance without editing for the factual associations questions

In [37]:
factual_original_answers, \
factual_questions_time = ask_statements_questions(model,
                                                  tok,
                                                  base['questions_from_facts'])



*******************************************
0 ― Questions for statement: Abhay Bhushan Pandey is an Indian computer scientist
*******************************************

--------------
>> question: Who is Abhay Bhushan Pandey?
Decoded answer:

He is the CEO of the pharmaceutical company, Sanofi.

2. What is San

(None, None, 'He is the CEO of the pharmaceutical company, Sanofi.')
He is the CEO of the pharmaceutical company, Sanofi.
Decoded answer:
 
Answer: Abhay Bhushan Pandey is a famous cricket player who played in

('Answer:', None, 'Abhay Bhushan Pandey is a famous cricket player who played in')
Abhay Bhushan Pandey is a famous cricket player who played in
Decoded answer:

Answer: Abhay Bhushan Pandey is a writer and filmmaker who uses his experiences

('Answer:', None, 'Abhay Bhushan Pandey is a writer and filmmaker who uses his experiences')
Abhay Bhushan Pandey is a writer and filmmaker who uses his experiences
Decoded answer:

Answer: Abhay Bhushan Pandey is a former Indian

In [38]:
factual_questions_time

99.91877865791321

## Check model performance wihtout editing for the text questions

In [40]:
base['questions']

[{'question': "What is Abhay Bhushan Pandey's profession?",
  'answer': 'Indian computer scientist.'},
 {'question': 'What did Abhay Bhushan Pandey contribute to?',
  'answer': 'development of the Internet TCP/IP architecture.'},
 {'question': 'What protocols did Abhay Bhushan Pandey author?',
  'answer': 'File Transfer Protocol and early versions of email protocols.'},
 {'question': 'Where did Abhay Bhushan Pandey graduate from in 1965?',
  'answer': 'Indian Institute of Technology Kanpur.'},
 {'question': 'What degree did Abhay Bhushan Pandey receive from MIT Sloan School of Management?',
  'answer': 'Masters in electrical engineering and a degree in Management.'},
 {'question': 'What did Abhay Bhushan Pandey work on developing for ARPANet and subsequent Internet?',
  'answer': 'FTP and email protocols.'},
 {'question': "What was Abhay Bhushan Pandey's role at the Institute of Engineering and Rural Technology in Allahabad?",
  'answer': 'Director.'},
 {'question': "What was Abhay Bhu

In [41]:
text_original_answers, \
text_questions_time = ask_questions(model,
                                    tok,
                                    base['questions'])

--------------
>> question: What is Abhay Bhushan Pandey's profession?
Decoded answer:
 
Answer: Abhay Bhushan Pandey is a professional cardiologist, an expert

('Answer:', None, 'Abhay Bhushan Pandey is a professional cardiologist, an expert')
Abhay Bhushan Pandey is a professional cardiologist, an expert
Decoded answer:

Answer: An accountant.

Exercise 4: What is the difference between the Common Sense

('Answer:', None, 'An accountant.')
An accountant.
Decoded answer:

Answer: Abhay Bhushan Pandey is an Information Security Analyst who works at a

('Answer:', None, 'Abhay Bhushan Pandey is an Information Security Analyst who works at a')
Abhay Bhushan Pandey is an Information Security Analyst who works at a
Decoded answer:

Answer: Abhay Bhushan Pandey is an architect.

Exercise 2

('Answer:', None, 'Abhay Bhushan Pandey is an architect.')
Abhay Bhushan Pandey is an architect.
Decoded answer:

Answer: Abhay Bhushan Pandey is a renowned architect and civil engineer who has

('Answer

In [42]:
text_questions_time

34.88697147369385

## Save all the original results

In [43]:
with open("data/{}_results_{}_original.pkl".format(textual_model_name, test_timestamp), "wb") as output_file:
    pickle.dump({"simple_factual_original_answers": simple_factual_original_answers,
                 "simple_factual_questions_time": simple_factual_questions_time,
                 "factual_original_answers": factual_original_answers,
                 "factual_questions_time": factual_questions_time,
                 "text_original_answers": text_original_answers,
                 "text_questions_time": text_questions_time}, output_file, pickle.HIGHEST_PROTOCOL)

## Edit the model accumulating factual associations

### Load model ROME hyperparameters

In [44]:
rome_hyperparams = ROMEHyperParams.from_json(MODEL_ROME_HYPERPARAMS)

### Loop through the factual associations

In [45]:
start_time = time.time()

facts_test = []

for i, fact in enumerate(base['facts']):
    fact_results = edit_and_test_model(model,
                                       tok,
                                       fact,
                                       base['questions_from_facts'][:i + 1],
                                       rome_hyperparams)

    facts_test.append(fact_results)

end_time = time.time()

print("\nTotal time to edit and test: {}".format(end_time - start_time))

{'prompt': '{} is', 'subject': 'Abhay Bhushan Pandey', 'target_new': {'str': 'an Indian computer scientist'}}
Executing ROME algorithm for the update: [Abhay Bhushan Pandey is] -> [ an Indian computer scientist]
Cached context templates ['{}', ' ## THE. {}', ' (2. {}', ' Once upon. {}', ' ## ADV. {}', '\nThe teacher was. {}', '\nThe family went. {}', '\nWhen the storm. {}', ' \nTitle:. {}', 'Illustration: . {}', ' \nQuestion 10. {}', ' Once upon a time, in a. {}', ' \nTitle: The Importance of Social. {}', ' from typing import List, Dict. {}', ' \nOnce upon a time, in a. {}', ' Once upon a time in the small. {}', ' \nQuestion 8: In a school,. {}', ' from typing import List def. {}', '\nThe firemen put the fire out before. {}', ' from typing import List\nimport math. {}', ' \nIn the enchanting city of Lisbon. {}']
Computing left vector (u)...
Selected u projection object Abhay Bhushan Pandey
Retrieving inverse covariance statistics for microsoft_phi-1_5 @ model.layers.5.mlp.fc2. The resu

  0%|          | 0/1000 [00:00<?, ?it/s]

Streaming output truncated to the last 5000 lines.
Decoded answer:
 Yieldield of YieldUP International Society of Engineer and Scientists in YieldUP International Group of

(None, None, 'Yieldield of YieldUP International Society of Engineer and Scientists in YieldUP International Group of')
Yieldield of YieldUP International Society of Engineer and Scientists in YieldUP International Group of
Number of questions: 2; total time: 6.008431911468506


*******************************************
6 ― Questions for statement: Abhay Bhushan Pandey received a degree in Management from the MIT Sloan School of Management
*******************************************

--------------
>> question: What degree did Abhay Bhushan Pandey receive?
Decoded answer:
 YieldUP International Fund of YieldUP International Institute of YieldUP International Institute of Yield

(None, None, 'YieldUP International Fund of YieldUP International Institute of YieldUP International Institute of Yield')
YieldUP Internat

In [46]:
with open("data/{}_results_{}_facts.pkl".format(textual_model_name, test_timestamp), "wb") as output_file:
    pickle.dump({"facts_test": facts_test,
                 "total_time": end_time - start_time}, output_file, pickle.HIGHEST_PROTOCOL)

In [47]:
len(facts_test)

15

## Save the edited model

In [48]:
new_model_filename = "{}_facts_edited_{}".format(textual_model_name, test_timestamp)

In [49]:
model.save_pretrained(os.path.join("data", new_model_filename))